# Model-agnostic Gateway

### Install libraries

In [1]:
!pip install -U langchain langchain-openai langgraph fastapi uvicorn

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Human in the loop

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command

@tool
def risky_operation(secret: str) -> str:
    """Perform a risky operation that requires manual approval."""
    return f"Executed risky operation with: {secret}"

tools = [risky_operation]
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

hitl = HumanInTheLoopMiddleware(
    interrupt_on={
        "risky_operation": {"allowed_decisions": ["approve", "edit", "reject"]}
    },
    description_prefix="Manual approval required for risky operation:"
)


checkpointer = MemorySaver()
agent = create_agent(
    model=model,
    tools=tools,
    middleware=[hitl],
    checkpointer=checkpointer,
    debug=True
)

config = {"configurable": {"thread_id": "hitl-demo-1"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": "Please run the risky operation with secret code $%45654@."}]},
    config=config,
)


[values] {'messages': [HumanMessage(content='Please run the risky operation with secret code $%45654@.', additional_kwargs={}, response_metadata={}, id='589244c7-9860-48fa-b68a-eca595510a73')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 60, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaJj7md4CRaAN2mcI1ju8uek8BJti', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--35ad04bd-5d01-4649-a64c-d8c583ffe3aa-0', tool_calls=[{'name': 'risky_operation', 'args': {'secret': '$%45654@'}, 'id': 'call_dK786IhVaO3Z4VssPOI1cM6y', 'type': 't

In [4]:
if "__interrupt__" in result:
    print("Interrupt detected!")
    decisions = [{"type": "approve"}]

    result = agent.invoke(
        Command(resume={"decisions": decisions}),
        config=config,
    )

Interrupt detected!
[values] {'messages': [HumanMessage(content='Please run the risky operation with secret code $%45654@.', additional_kwargs={}, response_metadata={}, id='589244c7-9860-48fa-b68a-eca595510a73'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 60, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaJj7md4CRaAN2mcI1ju8uek8BJti', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--35ad04bd-5d01-4649-a64c-d8c583ffe3aa-0', tool_calls=[{'name': 'risky_operation', 'args': {'secret': '$%45654@'}, 'id': 'call_dK786IhVaO3Z4VssPOI1cM6y', 'type': 'tool_call'}], us

### Model agnostic API gateway

In [5]:
#  Place the above code in a file app.py

from fastapi import FastAPI, Header
from pydantic import BaseModel
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI

class ChatRequest(BaseModel):
    message: str

class ChatResponse(BaseModel):
    provider: str
    model: str
    answer: str

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{message}")
])

def build_model(x_model: str):
    """
    x_model format:
      - 'openai:gpt-4o-mini'
    """
    if ":" in x_model:
        provider, model_name = x_model.split(":", 1)
    else:
        provider, model_name = "openai", x_model

    provider = provider.lower().strip()

    if provider == "openai":
        return provider, model_name, ChatOpenAI(model=model_name, temperature=0)

    #  if provider == "anthropic": # support for another LLM API provider
    #      return provider, model_name, ChatAnthropic(model=model_name, temperature=0)

    def _unknown(inputs: dict):
        return AIMessage(content=f"(unknown provider) Echo: {inputs.get('message','')}")
    return "unknown", x_model, RunnableLambda(_unknown)


app = FastAPI(title="Model-Agnostic LangChain Gateway")


@app.post("/chat", response_model=ChatResponse)
def chat_endpoint(
    req: ChatRequest,
    x_model: str = Header(default="openai:gpt-4o-mini", alias="X-Model"),
):
    provider, model_name, model = build_model(x_model)
    chain = prompt | model | StrOutputParser()
    answer: str = chain.invoke({"message": req.message})
    return ChatResponse(provider=provider, model=model_name, answer=answer)

1. Place the above code in a file `app.py`
2. Start server:
`uvicorn app:app --reload`

3. Send request:
```
curl -X POST 'http://127.0.0.1:8000/chat' \
  -H 'Content-Type: application/json' \
  -H 'X-Model: openai:gpt-5-mini' \
  -d '{"message":"Podaj 3 zalety Pythona."}'
```

```
curl -X POST 'http://127.0.0.1:8000/chat' \
  -H 'Content-Type: application/json' \
  -H 'X-Model: openai:gpt-4o-mini' \
  -d '{"message":"Podaj 3 zalety Pythona."}'
```